# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Import data from csv
data=pd.read_csv('WeatherPy/city_data.csv')
#Find components for heat map
locations=data[['Lat','Long']]
Humidity=data['Humidity']
data.head()

,City Name,Lat,Long,Max Temp,Cloudiness,Humidity,Wind Speed,Country
0,rikitea,-23.12,-134.97,73.33,100.0,84.0,20.69,PF
1,ancud,-41.87,-73.82,48.99,92.0,97.0,20.00,CL
2,qaanaaq,77.48,-69.36,25.86,98.0,77.0,9.10,GL
3,hobart,-42.88,147.33,48.20,99.0,66.0,12.75,AU
4,mataura,-46.19,168.86,42.01,54.0,87.0,1.41,NZ


In [4]:
#Display heat map
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [67]:
#Specify Data

#Not too windy
data_specific=data.loc[data['Wind Speed']<5]

#Not too hot
data_specific=data_specific.loc[data_specific['Max Temp']<90]

#Not too cold
data_specific=data_specific.loc[data_specific['Max Temp']>70]

#Not too cloudy
data_specific=data_specific.loc[data_specific['Cloudiness']<80]
data_specific=data_specific.reset_index(drop=True)
data_specific


,City Name,Lat,Long,Max Temp,Cloudiness,Humidity,Wind Speed,Country
0,gorontalo,0.54,123.06,75.20,78.0,88.0,2.55,ID
1,sao joao da barra,-21.64,-41.05,84.20,20.0,51.0,4.70,BR
2,puerto carreno,6.19,-67.49,89.10,23.0,61.0,0.69,CO
3,khajuraho,24.85,79.93,83.46,64.0,60.0,3.33,IN
4,shingu,33.73,135.98,73.06,45.0,89.0,4.27,JP


In [70]:
#Initialize hotel dataframe

hotel_df=pd.DataFrame({'Hotel Name':[],
                       'Hotel Lat':[],
                       'Hotel Lng':[]})

#Run api search for hotels near each city
for x in range(len(data_specific)):
    #build url to search
    lat=data_specific['Lat'][x]
    lng=data_specific['Long'][x]
    target_coordinates=f'{lat},{lng}'
    target_radius=5000
    target_type='lodging'

    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #Exception in case no hotel in range (unlikely but possible)
    try:
        response = requests.get(base_url, params=params)
        hotel_data = response.json()

        hotel_df.at[x,'Hotel Name']=hotel_data['results'][0]['name']
        hotel_df.at[x,'Hotel Lat']=hotel_data['results'][0]['geometry']['location']['lat']
        hotel_df.at[x,'Hotel Lng']=hotel_data['results'][0]['geometry']['location']['lng']
    except:
        hotel_df.at[x,'Hotel Name']='No Hotel Found'

#Remove rows if no hotel found
hotel_df=hotel_df.loc[hotel_df['Hotel Lat'].isnull()==False]
hotel_df.head()

,Hotel Name,Hotel Lat,Hotel Lng
0,Maqna Hotel by Prasanthi,0.537819,123.062642
1,Pousada Porto De Canoas,-21.640823,-41.052153
2,Casa Hotel Victoria,6.188846,-67.483564
3,The LaLiT Temple View Khajuraho,24.857518,79.921095
4,Shingu UI Hotel,33.722728,135.991572


In [76]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout=figure_layout)
locations_specific=data_specific[['Lat','Long']]
Humidity_specific=data_specific['Humidity']
heat_layer = gmaps.heatmap_layer(locations_specific, weights=Humidity_specific, 
                                 dissipating=False, max_intensity=max(Humidity_specific),
                                 point_radius=1)
fig.add_layer(heat_layer)
markers_coord=hotel_df[['Hotel Lat', 'Hotel Lng']]
markers=gmaps.marker_layer(markers_coord)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…